# 3 - Working with LBN

This tutorial was made in collaboration with Htet Aung Myin and Daniel Sun.

LBN, or Lorentz Boost Network is a type of machine learning model that uses a four-momenta vector as an input to build the model. 

LBN can be used to classify both a top tagging jet survey and a five tagger jet survey, but for the purposes of the tutorial, we will be focusing on classifying a top tagging jet survey.

## Package Dependencies

LBN is reliant on the following packages to run:<br>
Python3<br>
h5py<br>
numpy<br>
pandas<br>
cudatoolkit<br>
TensorFlow<br>

Make sure to download them using an installer such as pip or conda before working on LBN.

## Gathering All the Data

Import all of the necessary packages.

In [ ]:
import os
import h5py
import pandas as pd
from lbn import LBN, LBNLayer
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

Next, check the number of GPUs available in the computer to use.

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') #Gathers the list of GPUs available to use
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.set_memory_growth(physical_devices[0], True) #Selects the first available device (usually a computer will not have more than one GPU)

Open up all of the preprocessed data and store it in separate variables before closing the files; there should be one of x_train, y_train, x_test, and y_test before closing all files.

In [ ]:
h5f_X_train = h5py.File('../data/top-tagging/X_train.h5', 'r')
X_train = h5f_X_train['data'][:]
h5f_X_train.close()

h5f_y_train = h5py.File('../data/top-tagging/y_train.h5', 'r')
y_train = h5f_y_train['data'][:]
h5f_y_train.close()

h5f_X_test = h5py.File('../data/top-tagging/X_test.h5', 'r')
X_test = h5f_X_test['data'][:]
h5f_X_test.close()

h5f_y_test = h5py.File('../data/top-tagging/y_test.h5', 'r')
y_test = h5f_y_test['data'][:]
h5f_y_test.close()

Finally, get the dataset from zenodo and open up the file.

In [ ]:
f = pd.read_hdf('../data/top-tagging/test.h5','table')

In [ ]:
f

In [ ]:
len(f)

In [ ]:
X_test.shape #Checking shape of test set to see what values are needed for the input shape of the model

## Building the LBN Model

Now comes building the actual model. Start with establishing the input shape that you know from the shape of the X_test file.

In [ ]:
input_shape = (13,4)

inputs = keras.Input(shape=input_shape)

Now, make the actual LBN Layer to begin building the model. If you are curious as to why the layer is made in a specific way, here are the links to the repository for the basic LBN model and the paper to guide you.<br>
https://github.com/riga/LBN<br>
https://arxiv.org/pdf/1812.09722.pdf<br>
Page 3 describes the model shape in greater detail.

In [ ]:
x = LBNLayer(input_shape, 13, boost_mode=LBN.PAIRS, features=["E", "pt", "eta", "phi", "m", "pair_cos"])(inputs)

Next, add the Dense layers and the output layer to complete the creation of all layers for the model. The activation for the output layer should be sigmoid, as it is resulting in one node. If you make a model that has an output layer made up of more than one node, use softmax activation.

In [ ]:
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc1_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc2_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc3_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc4_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc5_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc6_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc7_elu')(x)
x = BatchNormalization()(x)
x = Dense(1024, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='elu', name='fc8_elu')(x)
x = BatchNormalization()(x)
outputs = Dense(1, kernel_initializer='lecun_uniform', kernel_regularizer='l2', activation='sigmoid', name='output_sigmoid')(x)

Make the model itself, along with the Adam optimizer with a learning rate of 0.00001, and compile the model.

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs, name="lbn-zenodo")
adam = Adam(lr=0.00001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

## Training the Model

Finally, begin training the model. If training each epoch takes too long, change the batch size, and make the number of epochs smaller to ensure you don't leave it running for 24 hours.

In [ ]:
history = model.fit(X_train, y_train, batch_size = 1024, epochs = 15, 
                    validation_split = 0.1, shuffle = True, callbacks = None,
                    use_multiprocessing=True, workers=4)

Make the method to plot the learning curve and the ROC curve.

In [15]:
import matplotlib.pyplot as plt
def learningCurve(history):
    plt.figure(figsize=(10,8))
    plt.plot(history.history['loss'], linewidth=1)
    plt.plot(history.history['val_loss'], linewidth=1)
    plt.title('Model Loss over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['training sample loss','validation sample loss'])
    plt.savefig('zenodo-l2_learning_curve.pdf')
    plt.show()
    plt.close()

In [16]:
def makeRoc(features_val, labels_val, labels, model, outputDir='', outputSuffix=''):
    from sklearn.metrics import roc_curve, auc
    labels_pred = model.predict(features_val)
    df = pd.DataFrame()
    fpr = {}
    tpr = {}
    auc1 = {}
    plt.figure(figsize=(10,8))       
    for i, label in enumerate(labels):
        df[label] = labels_val[:,i]
        df[label + '_pred'] = labels_pred[:,i]
        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])
        auc1[label] = auc(fpr[label], tpr[label])
        plt.plot(fpr[label],tpr[label],label='%s tagger, AUC = %.1f%%'%(label.replace('j_',''),auc1[label]*100.))
    plt.plot([0, 1], [0, 1], lw=1, color='black', linestyle='--')
    #plt.semilogy()
    plt.xlabel("Background Efficiency")
    plt.ylabel("Signal Efficiency")
    plt.xlim([-0.05, 1.05])
    plt.ylim(0.001,1.05)
    plt.grid(True)
    plt.legend(loc='lower right')
    plt.figtext(0.25, 0.90,'LBN ROC Curve',fontweight='bold', wrap=True, horizontalalignment='right', fontsize=14)
    #plt.figtext(0.35, 0.90,'preliminary', style='italic', wrap=True, horizontalalignment='center', fontsize=14) 
    #plt.savefig('%sROC_%s.pdf'%(outputDir, outputSuffix))
    return labels_pred

Use the methods to plot the curves.

In [ ]:
learningCurve(history)

In [ ]:
labels_pred = model.predict(X_test)

In [ ]:
y_pred = makeRoc(X_test, y_test, ['is_signal_new'], model, outputSuffix='lbn-zenodo-l2')

Congrats! You have just successfully built a functioning LBN model!